# Probe Designer for Isoforms


## Environment


In [1]:
# basci env
import os
import sys
import pandas as pd
import time
import json
from tqdm import tqdm

# data process of file from ncbi
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import MeltingTemp as mt

# get gene data from ncbi
from Bio import Entrez

# blast and xml file process
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# add package to sys var
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
try:
    sys.path.append("../lib")
except:
    pass

# dir
project_name = "2023.11.20_Isoform_trials"
dataset_path = "/home/akikawa/repos/probe_designer/dataset/"

os.makedirs(f"{dataset_path}/{project_name}", exist_ok=True)
os.chdir(f"{dataset_path}/{project_name}")

current_time = time.localtime()
formatted_time = time.strftime("%Y%m%d_%H%M%S", current_time)
tmp = f"./trials/{formatted_time}/"
output = f"./trials/{formatted_time}/"
pre_binding_dir = f"{tmp}/pre_binding/"
os.makedirs(tmp, exist_ok=True)

# basic variables
gene_name_list_tosearch = "gene_name_list_tosearch.txt"
pre_binding_file_suffix = "_pre_binding.fasta"
total_pre_binding_file_name = "_total_pre_binding.fasta"

# tmp file
gene_name_list_file = "1_gene_name_list.txt"
gene_id_name_file = "2_id_list.txt"
gene_seq_in_file = "3_gene_seq_in_file.gb"
pre_binding_num_file = "4_pre_binding_num.json"
blast_results_file = "5_blast_results.xml"

In [2]:
import os


workdir = r'./dataset/2023.11.22_Isoform_trials/'

## Get seq file of each gene from ensembl dataset


In [3]:
from tqdm import tqdm
import requests

species = "human"
gene = "BRCA1"
seq_type = "cds"

def get_seqs(gene, species, seq_type):
    lookup_url = f"http://rest.ensembl.org/lookup/symbol/{species}/{gene}?content-type=application/json"
    gene_id = requests.get(url=lookup_url).json()["id"]

    transcripts_url = f"http://rest.ensembl.org/overlap/id/{gene_id}?feature=transcript;content-type=application/json"
    transcripts = requests.get(url=transcripts_url).json()

    # Get sequences for each transcript
    sequences = {}
    for transcript in tqdm(transcripts, desc=f'Gene_{gene}'):
        try:
            seq_name = "_".join([transcript["id"], transcript["external_name"], transcript['biotype']])
            seq_url = f"http://rest.ensembl.org/sequence/id/{transcript['id']}?type={seq_type};content-type=application/json"
            seq_response = requests.get(seq_url).json()
            sequences[seq_name] = seq_response["seq"]
        except: continue

    return sequences

sequences = get_seqs(gene, species, seq_type)    

Gene_BRCA1: 100%|██████████| 56/56 [00:24<00:00,  2.27it/s]


In [6]:
from tqdm import tqdm
from lib.search_binding import step_by_step


def calculate_optimal_alignment_similarity(sub1, sub2):
    max_similarity = 0
    for shift in range(-len(sub1) + 1, len(sub1)):
        shifted_similarity = sum((sub1[i] == sub2[i - shift]) for i in range(len(sub1)) if 0 <= i - shift < len(sub2))
        max_similarity = max(max_similarity, shifted_similarity)
    return max_similarity

def find_most_distinctive_substrings(sequences_dict, substring_length=40, top_n=10):
    if any(len(seq) < substring_length for seq in sequences_dict.values()):
        raise ValueError("One or more sequences are shorter than the specified substring length.")
    
    all_substrings = {}
    for name, seq in sequences_dict.items():
        _, _, selected_substrings, _ = step_by_step(
        seq,
        BDS_len=40,
        BDS_num=50,
        min_gap=1,
        better_gap=40,
        gene=name,
        G_min=0.25,
        G_max=0.7,
        G_consecutive=5,
        Tm_low=50,
        Tm_high=65,
        show_process=False, 
        warn=False
        )
        
        all_substrings[name] = selected_substrings

    top_substrings = {}
    for name, substrings in all_substrings.items():
        distinctiveness_scores = {}
        for sub in tqdm(substrings, desc=name):
            distinctiveness_scores[sub] = min(calculate_optimal_alignment_similarity(sub, other_sub)
                                              for other_name, other_substrings in all_substrings.items()
                                              if other_name != name
                                              for other_sub in other_substrings)

        top_n_substrings = sorted(distinctiveness_scores, key=distinctiveness_scores.get)[:top_n]
        top_substrings[name] = top_n_substrings

    return top_substrings

distinctive_substrings = find_most_distinctive_substrings(sequences, substring_length=40)


Gene ENST00000461221_BRCA1-204_nonsense_mediated_decay: 	No valid positions, please loosen your threshold conditions.
Gene ENST00000700183_BRCA1-237_nonsense_mediated_decay: 	No valid positions, please loosen your threshold conditions.
Gene ENST00000492859_BRCA1-218_nonsense_mediated_decay: 	No valid positions, please loosen your threshold conditions.
Gene ENST00000461798_BRCA1-206_nonsense_mediated_decay: 	No valid positions, please loosen your threshold conditions.


ENST00000493795_BRCA1-219_protein_coding: 100%|██████████| 43/43 [00:16<00:00,  2.53it/s]
ENST00000461221_BRCA1-204_nonsense_mediated_decay: 0it [00:00, ?it/s]
ENST00000700182_BRCA1-236_protein_coding: 100%|██████████| 20/20 [00:08<00:00,  2.47it/s]
ENST00000700183_BRCA1-237_nonsense_mediated_decay: 0it [00:00, ?it/s]
ENST00000492859_BRCA1-218_nonsense_mediated_decay: 0it [00:00, ?it/s]
ENST00000642945_BRCA1-229_nonsense_mediated_decay: 100%|██████████| 11/11 [00:04<00:00,  2.46it/s]
ENST00000461798_BRCA1-206_nonsense_mediated_decay: 0it [00:00, ?it/s]


In [7]:
distinctive_substrings

{'ENST00000497488_BRCA1-222_protein_coding': ['AGGCTGAGGAGGAAGTCTTCTACCAGGCATATTCATGCGC',
  'GGAGGCTCTAGGTTTTGTCTATCATCTCAGTTCAGAGGCA',
  'ACAGCCTATGGGAAGTAGTCATGCATCTCAGGTTTGTTCT',
  'CAGCCTATGGGAAGTAGTCATGCATCTCAGGTTTGTTCTG',
  'GCGTCCAGAAAGGAGAGCTTAGCAGGAGTCCTAGCCCTTT',
  'TTCACCCATACACATTTGGCTCAGGGTTACCGAAGAGGGG',
  'TCACCCATACACATTTGGCTCAGGGTTACCGAAGAGGGGC',
  'CACCCATACACATTTGGCTCAGGGTTACCGAAGAGGGGCC',
  'ACCCATACACATTTGGCTCAGGGTTACCGAAGAGGGGCCA',
  'CCCATACACATTTGGCTCAGGGTTACCGAAGAGGGGCCAA'],
 'ENST00000489037_BRCA1-216_protein_coding': ['AGGCTGAGGAGGAAGTCTTCTACCAGGCATATTCATGCGC',
  'GGAGGCTCTAGGTTTTGTCTATCATCTCAGTTCAGAGGCA',
  'ACAGCCTATGGGAAGTAGTCATGCATCTCAGGTTTGTTCT',
  'CAGCCTATGGGAAGTAGTCATGCATCTCAGGTTTGTTCTG',
  'GCGTCCAGAAAGGAGAGCTTAGCAGGAGTCCTAGCCCTTT',
  'TTCACCCATACACATTTGGCTCAGGGTTACCGAAGAGGGG',
  'TCACCCATACACATTTGGCTCAGGGTTACCGAAGAGGGGC',
  'CACCCATACACATTTGGCTCAGGGTTACCGAAGAGGGGCC',
  'ACCCATACACATTTGGCTCAGGGTTACCGAAGAGGGGCCA',
  'CCCATACACATTTGGCTCAGGGTTACCGAAGAGG

## Binding site Searcher


In [ ]:
from lib.search_binding import step_by_step, find_max_min_difference_fixed_length_subsequence, gb_extract

# Initiation of array
binding_site_FOIs = [
    "accession",
    "gene_name",
    "mol_type",
    "organism",
    "pos_on_seq",
    "binding",
    "Tm_l",
    "Tm_r",
    "wanted",
]
align_FOIs = ["align_num", "align_accession", "align_descrip", "plus/minus"]
FOI = pd.DataFrame(columns=binding_site_FOIs + align_FOIs)

# Search binding sites on mRNA sequence
file_in = tmp + gene_seq_in_file
file_out_dir = pre_binding_dir
try:
    os.mkdir(file_out_dir)
except:
    pass

pre_binding_num = {}

# initialization of file
with open(file_out_dir + total_pre_binding_file_name, "w") as handle:
    handle.write("")

for gene_seq_gb in SeqIO.parse(tmp + gene_seq_in_file, "genbank"):
    id, gene_name, mol_type, organism, seq = gb_extract(gene_seq_gb)

    Tm_l, Tm_r, selected_substrings, pos_on_seq = step_by_step(
        seq,
        BDS_len=40,
        BDS_num=50,
        min_gap=1,
        better_gap=40,
        gene=gene_name,
        G_min=0.25,
        G_max=0.7,
        G_consecutive=5,
        Tm_low=50,
        Tm_high=65,
    )
    
    record_list = []
    for i, pre_binding_tmp in enumerate(selected_substrings):
        record_list.append(
            SeqRecord(
                Seq(pre_binding_tmp),
                id="pre_binding" + str(i),
                description="|".join([id, gene_name, organism, mol_type]),
            )
        )

    # add information about binding sites to FOI
    add = pd.DataFrame(
        {
            "accession": [id] * len(selected_substrings),
            "gene_name": [gene_name] * len(selected_substrings),
            "mol_type": [mol_type] * len(selected_substrings),
            "organism": [organism] * len(selected_substrings),
            "binding": selected_substrings,
            "Tm_l": Tm_l,
            "Tm_r": Tm_r,
            "pos_on_seq": pos_on_seq,
        }
    )
    FOI = pd.concat([FOI, add], ignore_index=True)

    file_out = file_out_dir + gene_name + pre_binding_file_suffix
    # write pre_binding to files
    with open(file_out, "w") as f:
        for new_record in record_list:
            SeqIO.write(new_record, f, "fasta")
    with open(file_out_dir + total_pre_binding_file_name, "a") as handle:
        for new_record in record_list:
            SeqIO.write(new_record, handle, "fasta")

    # record the num of pre_binding for each gene
    pre_binding_num[f"{id}_{gene_name}"] = len(selected_substrings)

with open(tmp + pre_binding_num_file, "w") as f:
    json.dump(pre_binding_num, f)

position_searching_CTRB1: 100%|██████████| 656/656 [00:00<00:00, 11751.96it/s]

condition for gene CTRB1 too harsh, loose to get better results
[2, 12, 21, 33, 42, 51, 67, 119, 129, 138, 151, 160, 173, 185, 194, 214, 234, 243, 252, 265, 274, 285, 294, 303, 313, 323, 333, 342, 353, 362, 371, 422, 433, 442, 451, 460, 484, 499, 508, 519, 528, 537, 546, 555, 566, 575, 586, 595, 604, 623, 639, 648]


## Blast and extract blast results

NCBIXML: https://homolog.us/Biopython/Bio.Blast.NCBIXML.html#read/0

BlastRecord: https://biopython.org/docs/1.75/api/Bio.Blast.Record.html

XMLReader: https://codebeautify.org/xmlviewer#


In [ ]:
with open(file_out_dir + total_pre_binding_file_name, "r") as f:
    fasta_string = f.read()
txid = [2697049]  # organism

# Submit BLAST search and get handle object
handle = NCBIWWW.qblast(
    program="blastn",
    megablast="yes",
    database="refseq_rna",
    sequence=fasta_string,
    url_base="https://blast.ncbi.nlm.nih.gov/Blast.cgi",
    format_object="Alignment",
    format_type="Xml",
)

# read handle object and save to a file
with open(tmp + blast_results_file, "w") as f:
    f.write(handle.read())

In [94]:
# Extract interested information from blast_results
align_num = []

# read the id/plus-minus part/align_num
with open(tmp + blast_results_file, "r") as blast_output:
    blast_records = NCBIXML.parse(blast_output)
    loca = 0
    for blast_record in blast_records:
        align_accession = []
        align_descrip_list = []
        # get align num of each binding site
        length = len(blast_record.alignments)
        align_num.append(length)
        for i in range(length):
            descrip = blast_record.descriptions[i].title.split("|")
            # get accession and descrip of each align seq
            align_accession.append(descrip[3])
            align_descrip_list.append(descrip[-1])
        FOI.loc[loca, "align_accession"] = "|".join(str(_) for _ in align_accession)

        # add align_descrip to df
        FOI.loc[loca, "align_descrip"] = "|".join(str(_) for _ in align_descrip_list)

        # get plus/minus of each align seq
        p_m = [blast_record.alignments[_].hsps[0].frame[1] for _ in range(length)]

        # add plus/minus to df
        try:
            FOI.loc[loca, "plus/minus"] = ",".join([str(_) for _ in p_m])
        except:
            FOI.loc[loca, "plus/minus"] = "NAN"

        loca += 1

FOI["align_num"] = align_num

## Select wanted binding site


In [95]:
FOI["wanted"] = [True] * len(FOI)

In [96]:
# sieve for the suitable binding site
gene_name_list = [_.upper() for _ in gene_name_list]
gene_name_list_out = [i for i in gene_name_list]
for i in range(len(FOI)):
    # check gene_name
    gene_name = FOI.loc[i, "gene_name"]
    if gene_name.upper() not in gene_name_list:
        FOI.loc[i, "wanted"] = False
    else:
        try:
            gene_name_list_out.remove(gene_name)
        except:
            pass

    # check DNA or mRNA type
    if FOI.loc[i, "wanted"] == True:
        if FOI.loc[i, "mol_type"] != "mRNA":
            FOI.loc[i, "wanted"] = False

    # check gene_organism name
    if FOI.loc[i, "wanted"] == True:
        spe_ori, gene_ori = FOI.loc[i, "organism"], FOI.loc[i, "gene_name"]
        descrip = FOI.loc[i, "align_descrip"].split("|")
        for des in descrip:
            if gene_ori not in des and spe_ori in des:
                FOI.loc[i, "wanted"] = False
                break

    # check plus/minus
    if FOI.loc[i, "wanted"] == True:
        if pd.isnull(FOI.loc[i, "plus/minus"]):
            FOI.loc[i, "wanted"] = False
        else:
            pm_list = FOI.loc[i, "plus/minus"].split(",")
            if "-1" not in pm_list:
                FOI.loc[i, "wanted"] = False

# write the whole information of interest to a excel file in tmp dir
FOI.to_excel(tmp + "probes_sieve.xlsx")

out_tmp = FOI[FOI["wanted"] == True]
output_df = pd.DataFrame()
for gene in out_tmp.gene_name.unique():
    pos_of_True = list(out_tmp[out_tmp.gene_name == gene]["pos_on_seq"])
    best_pos = find_max_min_difference_fixed_length_subsequence(
        pos_of_True,
        length=3,
        min_gap=40,
        better_gap=80,
        gene=gene,
    )
    out_subset = out_tmp[out_tmp.gene_name == gene]
    out_subset = out_subset[out_subset["pos_on_seq"].isin(best_pos)]
    output_df = pd.concat([output_df, out_subset])

# write the output to a xlsx file
output_df.to_excel(output + "probes_wanted.xlsx")

condition for gene CTRB1 too harsh, loose to get better results
[89, 99, 108, 120, 129]
